# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **Air Quality** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Download model and batch inference data
2. Make predictions, generate PNG for forecast
3. Store predictions in a monitoring feature group adn generate PNG for hindcast

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Read the API keys and configuration variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

In [ ]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
from mlfs.airquality import util

In [ ]:
today = datetime.datetime.now() - datetime.timedelta(0)
tomorrow = today + datetime.timedelta(days = 1)
today

## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Feature Store </span>

In [ ]:
project = hopsworks.login()
fs = project.get_feature_store() 

secrets = hopsworks.get_secrets_api()
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)
country=location['country']
city=location['city']
street=location['street']

## <span style="color:#ff5f27;">🪝 Download the model from Model Registry</span>

In [ ]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="air_quality_xgboost_model",
    # version=1, no version means the latest version
)

fv = retrieved_model.get_feature_view()

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

In [ ]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

## <span style="color:#ff5f27;">✨ Get Weather Forecast Features with Feature View   </span>



In [ ]:
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
batch_data = weather_fg.filter(weather_fg.date >= today).read()
batch_data

## <span style="color:#ff5f27;">🔄 Get Lagged PM2.5 Features</span>

To make accurate predictions with our model (which includes lagged features), we need to fetch the most recent 3 days of PM2.5 measurements.

In [ ]:
# Get the air quality feature group to fetch recent PM2.5 values for lagged features
air_quality_fg_inference = fs.get_feature_group(name='air_quality', version=2)

# Get the most recent 3 days of PM2.5 data for lagged features
from datetime import timedelta

start_lag_date = today - timedelta(days=3)
end_lag_date = today - timedelta(days=1)

# Fetch recent PM2.5 values
recent_aq_query = air_quality_fg_inference.select(['date', 'pm25']) \
    .filter(air_quality_fg_inference.country == country) \
    .filter(air_quality_fg_inference.city == city) \
    .filter(air_quality_fg_inference.street == street) \
    .filter(air_quality_fg_inference.date >= start_lag_date) \
    .filter(air_quality_fg_inference.date <= end_lag_date)

try:
    recent_pm25_df = recent_aq_query.read()
    recent_pm25_df = recent_pm25_df.sort_values('date')
    
    if len(recent_pm25_df) >= 3:
        # Get the last 3 PM2.5 values for lagged features
        pm25_lags = recent_pm25_df['pm25'].tail(3).tolist()
        pm25_lag_1d = pm25_lags[-1]  # 1 day ago
        pm25_lag_2d = pm25_lags[-2]  # 2 days ago
        pm25_lag_3d = pm25_lags[-3]  # 3 days ago
        
        print(f"✅ Fetched lagged PM2.5 values:")
        print(f"   1 day ago: {pm25_lag_1d}")
        print(f"   2 days ago: {pm25_lag_2d}")
        print(f"   3 days ago: {pm25_lag_3d}")
    else:
        print(f"⚠️ Warning: Only found {len(recent_pm25_df)} days of data. Using zeros for missing lags.")
        pm25_lag_1d = 0
        pm25_lag_2d = 0
        pm25_lag_3d = 0
except Exception as e:
    print(f"⚠️ Error fetching lagged features: {e}")
    print("   Using zeros for lagged features.")
    pm25_lag_1d = 0
    pm25_lag_2d = 0
    pm25_lag_3d = 0

### <span style="color:#ff5f27;">🤖 Making the predictions</span>

**Note on Multi-Day Forecasting with Lagged Features:**  
- **Day 1:** Uses real PM2.5 values from the previous 3 days (most accurate)
- **Day 2+:** Uses a mix of real and predicted PM2.5 values as lagged features (rolling forecast)  
- Prediction accuracy typically decreases for days further into the future

In [ ]:
# Add lagged features to batch data
# For the first forecast day, we use real lagged values
# For subsequent days, we use predictions as pseudo-lags (rolling forecast)
batch_data = batch_data.reset_index(drop=True)
predictions = []

for i in range(len(batch_data)):
    # Prepare features for this prediction
    features_dict = {
        'temperature_2m_mean': batch_data.loc[i, 'temperature_2m_mean'],
        'precipitation_sum': batch_data.loc[i, 'precipitation_sum'],
        'wind_speed_10m_max': batch_data.loc[i, 'wind_speed_10m_max'],
        'wind_direction_10m_dominant': batch_data.loc[i, 'wind_direction_10m_dominant'],
    }
    
    # Add lagged features
    if i == 0:
        # First day: use real historical lags
        features_dict['pm25_lag_1d'] = pm25_lag_1d
        features_dict['pm25_lag_2d'] = pm25_lag_2d
        features_dict['pm25_lag_3d'] = pm25_lag_3d
    elif i == 1:
        # Second day: use 1 prediction, 2 real lags
        features_dict['pm25_lag_1d'] = predictions[0]
        features_dict['pm25_lag_2d'] = pm25_lag_1d
        features_dict['pm25_lag_3d'] = pm25_lag_2d
    elif i == 2:
        # Third day: use 2 predictions, 1 real lag
        features_dict['pm25_lag_1d'] = predictions[1]
        features_dict['pm25_lag_2d'] = predictions[0]
        features_dict['pm25_lag_3d'] = pm25_lag_1d
    else:
        # Fourth day onwards: all predictions
        features_dict['pm25_lag_1d'] = predictions[i-1]
        features_dict['pm25_lag_2d'] = predictions[i-2]
        features_dict['pm25_lag_3d'] = predictions[i-3]
    
    # Make prediction
    feature_vector = pd.DataFrame([features_dict])
    pred = retrieved_xgboost_model.predict(feature_vector)[0]
    predictions.append(pred)

batch_data['predicted_pm25'] = predictions
print(f"\n✅ Made {len(predictions)} predictions using lagged features")
batch_data

In [ ]:
batch_data.info()

### <span style="color:#ff5f27;">🤖 Saving the predictions (for monitoring) to a Feature Group</span>

In [ ]:
batch_data['street'] = street
batch_data['city'] = city
batch_data['country'] = country
# Fill in the number of days before the date on which you made the forecast (base_date)
batch_data['days_before_forecast_day'] = range(1, len(batch_data)+1)
batch_data = batch_data.sort_values(by=['date'])
batch_data

In [ ]:
batch_data.info()

### Create Forecast Graph
Draw a graph of the predictions with dates as a PNG and save it to the github repo
Show it on github pages

In [ ]:

pred_file_path = f"{root_dir}/docs/air-quality/assets/img/pm25_forecast.png"
plt = util.plot_air_quality_forecast(city, street, batch_data, pred_file_path)

plt.show()

In [ ]:
# Get or create feature group
monitor_fg = fs.get_or_create_feature_group(
    name='aq_predictions',
    description='Air Quality prediction monitoring',
    version=1,
    primary_key=['city','street','date','days_before_forecast_day'],
    event_time="date"
)

In [ ]:
monitor_fg.insert(batch_data, wait=True)

In [ ]:
# We will create a hindcast chart for  only the forecasts made 1 day beforehand
monitoring_df = monitor_fg.filter(monitor_fg.days_before_forecast_day == 1).read()
monitoring_df

In [ ]:
air_quality_fg = fs.get_feature_group(name='air_quality', version=2)  # Using version 2 with lagged features
air_quality_df = air_quality_fg.read()
air_quality_df

In [ ]:
outcome_df = air_quality_df[['date', 'pm25']]
preds_df =  monitoring_df[['date', 'predicted_pm25']]

hindcast_df = pd.merge(preds_df, outcome_df, on="date")
hindcast_df = hindcast_df.sort_values(by=['date'])

# If there are no outcomes for predictions yet, generate some predictions/outcomes from existing data
if len(hindcast_df) == 0:
    hindcast_df = util.backfill_predictions_for_monitoring(weather_fg, air_quality_df, monitor_fg, retrieved_xgboost_model)
hindcast_df

### Plot the Hindcast comparing predicted with forecasted values (1-day prior forecast)

__This graph will be empty to begin with - this is normal.__

After a few days of predictions and observations, you will get data points in this graph.

In [ ]:
hindcast_file_path = f"{root_dir}/docs/air-quality/assets/img/pm25_hindcast_1day.png"
plt = util.plot_air_quality_forecast(city, street, hindcast_df, hindcast_file_path, hindcast=True)
plt.show()

### Upload the prediction and hindcast dashboards (png files) to Hopsworks


In [ ]:
dataset_api = project.get_dataset_api()
str_today = today.strftime("%Y-%m-%d")
if dataset_api.exists("Resources/airquality") == False:
    dataset_api.mkdir("Resources/airquality")
dataset_api.upload(pred_file_path, f"Resources/airquality/{city}_{street}_{str_today}", overwrite=True)
dataset_api.upload(hindcast_file_path, f"Resources/airquality/{city}_{street}_{str_today}", overwrite=True)

proj_url = project.get_url()
print(f"See images in Hopsworks here: {proj_url}/settings/fb/path/Resources/airquality")

---